In [ ]:
from threading import Thread
import urllib
import json

symbols = "AAPL WEED.TO MSFT GOOG NFLX RHT.V ACB.TO TSLA".split()

def options(ur):
    htmlfile = urllib.request.urlopen("http://query2.finance.yahoo.com/v7/finance/options/" + ur + "?symbol=" + ur + "&straddle='true'")
    htmltext = htmlfile.read()
    print(htmltext)
    

threadlist=[]

for i in symbols:
    t = Thread(target=options, args=(i, ))
    t.start()
    threadlist.append(t)

for b in threadlist:
    b.join()

In [3]:
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials

json_key = json.load(open('creds.json'))

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('creds.json', scope) # get email and key from creds

file = gspread.authorize(credentials) # authenticate with Google
sheet = file.open("Watchlist").sheet1 # open sheet

all_cells = sheet.range('A2:A251')

stocks = []
for cell in all_cells:
    stocks.append(cell.value)

In [ ]:
import MySQLdb
myConnection = MySQLdb.connect( host='PythonWebscrape.db.3267106.3eb.hostedresource.net', user='PythonWebscrape', passwd='Doomer420!', db='PythonWebscrape' )
myConnection.close()

In [1]:
import urllib
import requests
import threading
import json
import pandas as pd
import time
from pandas.io.json import json_normalize

from sqlalchemy import create_engine

server = 'constantone.database.windows.net'
database = 'Hedge Fund'
username = 'felix.jc@gmail.com@constantone'
password = 'Doomer420!'
driver= '{ODBC Driver 13 for SQL Server}'

import urllib
quoted = urllib.parse.quote_plus('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)


engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))
def stock(ur):
    url = "https://www.alphavantage.co/query?symbol=" + ur + "&outputsize=full&function=TIME_SERIES_DAILY_ADJUSTED&apikey=SOLA8IKUMXUS875X"
#    htmljson = requests.get(url).json()
#    print(htmljson['Meta Data'])
#    if 'Time Series (Daily)' in htmljson.keys():
#        htmljson = pd.DataFrame.from_dict(htmljson['Time Series (Daily)'])
#    for key, value in htmljson.items():
#        print("Key:")
#        print(key)
    try:
        htmlfile = urllib.request.urlopen(url)
        htmltext = htmlfile.read()
    except:
        htmltext = ""
    
    return htmltext
    #return htmltext


    
threads = []


df = pd.DataFrame()
stockCount = 0
for symbol in stocks:
    stockCount += 1
    t = stock(symbol)
    threads.append(t)
    data = json.loads(t)
    
    if 'Time Series (Daily)' in data.keys():
        for key, value in data['Time Series (Daily)'].items():
            data['Time Series (Daily)'][key].update([('symbol',symbol), ('date', key)])
            keys = []
            values = []
            for key1, value1 in data['Time Series (Daily)'].get(key).items():
                keys.append(key1)
                values.append(value1)
                
            t1 = json_normalize(data = data['Time Series (Daily)'].get(key), errors='ignore')
            t1.to_sql('Stock History', engine, if_exists='append', index=False)
    else:
        time.sleep(60)
    
           
    print(stockCount)


NameError: name 'stocks' is not defined

In [ ]:
import pyodbc
server = 'constantone.database.windows.net'
database = 'Hedge Fund'
username = 'felix.jc@gmail.com@constantone'
password = 'Doomer420!'
driver= '{ODBC Driver 13 for SQL Server}'
cnxn = pyodbc.connect('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

import urllib
quoted = urllib.parse.quote_plus('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)


engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

cursor = engine.cursor()


In [ ]:
import urllib
import requests
import threading
from threading import Thread
import json
import pandas as pd
import time
from pandas.io.json import json_normalize

from sqlalchemy import create_engine

server = 'constantone.database.windows.net'
database = 'Hedge Fund'
username = 'felix.jc@gmail.com@constantone'
password = 'Doomer420!'
driver= '{ODBC Driver 13 for SQL Server}'

import urllib
quoted = urllib.parse.quote_plus('DRIVER='+driver+';SERVER='+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password)


engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted), pool_size=30, max_overflow=0, pool_reset_on_return='commit', pool_timeout=60)

def ema(symbol, period):
    url ="https://www.alphavantage.co/query?function=EMA&symbol=" + symbol + "&interval=daily&time_period=" + period +"&series_type=open&apikey=SOLA8IKUMXUS875X"
    
    try:
        htmlfile = urllib.request.urlopen(url)
        htmltext = htmlfile.read()
#         print(htmltext)
    except:
        htmltext = ""
    
    return htmltext

def stock(ur):
    url = "https://www.alphavantage.co/query?symbol=" + ur + "&outputsize=full&function=TIME_SERIES_DAILY_ADJUSTED&apikey=SOLA8IKUMXUS875X"

    try:
        htmlfile = urllib.request.urlopen(url)
        htmltext = htmlfile.read()
    except:
        htmltext = ""
    
    return htmltext

    
threadlist=[]
threadlistEMA=[]

threadLimiter = threading.BoundedSemaphore(25)

class EncodeThread(threading.Thread):

    def run(self):
        threadLimiter.acquire()
        try:
           print(0) 
        finally:
            threadLimiter.release()
            
def TimeSeriesPull(data, key, value,symbol):
    count=0
       
    data['Time Series (Daily)'][key].update([('symbol',symbol), ('date', key)])
    keys = []
    values = []
    for key1, value1 in data['Time Series (Daily)'].get(key).items():
        keys.append(key1)
        values.append(value1)

    t1 = json_normalize(data = data['Time Series (Daily)'].get(key), errors='ignore')
    t1.to_sql('Stock History', engine, if_exists='append', index=False)


def TimeSeriesPullEMA(data, key, value, symbol):
    count = 0
    
    data['Technical Analysis: EMA'][key].update([('symbol',symbol), ('date', key)])
    keys = []
    values = []
    for key1, value1 in data['Technical Analysis: EMA'].get(key).items():
        keys.append(key1)
        values.append(value1)

    t2 = json_normalize(data['Technical Analysis: EMA'].get(key), errors='ignore')
    t2.to_sql(datePeriod + ' Day EMA', engine, if_exists='append', index=False)
 
                
df = pd.DataFrame()
stockCount = 0
for symbol in stocks:
    stockCount += 1
    
    for datePeriod in ["12","26"]:
        try:
            data = json.loads(ema(symbol, datePeriod))
        except:
            time.sleep(60)
            
        print(data.keys())
        if 'Technical Analysis: EMA' in data.keys():
            for key, value in data['Technical Analysis: EMA'].items():
                while threading.active_count() > 29:
                    time.sleep(0.5)
                t = Thread(target=TimeSeriesPullEMA, args=(data,key,value,symbol))
                t.start()
                threadlistEMA.append(t)
        else:            
            

    print(stockCount)


dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
1
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
2
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
3
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
4
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
5
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
6
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
7
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
8
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
9
dict_keys(['Meta Data', 'Technical Analysis: E

Exception in thread Thread-157562:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\engine\base.py", line 1182, in _execute_context
    context)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 470, in do_execute
    cursor.execute(statement, parameters)
pyodbc.Error: ('08S01', '[08S01] [Microsoft][ODBC Driver 13 for SQL Server]TCP Provider: An existing connection was forcibly closed by the remote host.\r\n (10054) (SQLExecDirectW)')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-18-a3cb542363a4>", line 87, in TimeSeriesPullEMA
    t2.to_sql(datePeriod + ' Day EMA', engine, if_exists='append', index=False)
  File

dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
37
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
38
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
39
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
40
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
41
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
42
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
43
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
44
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
45
dict_keys(['Meta Data', 'Technical An

Exception in thread Thread-191879:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\engine\base.py", line 1182, in _execute_context
    context)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 470, in do_execute
    cursor.execute(statement, parameters)
pyodbc.Error: ('08S01', '[08S01] [Microsoft][ODBC Driver 13 for SQL Server]TCP Provider: An existing connection was forcibly closed by the remote host.\r\n (10054) (SQLExecDirectW)')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-18-a3cb542363a4>", line 87, in TimeSeriesPullEMA
    t2.to_sql(datePeriod + ' Day EMA', engine, if_exists='append', index=False)
  File

dict_keys(['Meta Data', 'Technical Analysis: EMA'])
47
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
48
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
49
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
50
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
51
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
52
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
53
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
54
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
55
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical An

...

86
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
87
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
88
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])


Exception in thread Thread-524375:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\engine\base.py", line 1182, in _execute_context
    context)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 470, in do_execute
    cursor.execute(statement, parameters)
pyodbc.Error: ('08S01', '[08S01] [Microsoft][ODBC Driver 13 for SQL Server]TCP Provider: An existing connection was forcibly closed by the remote host.\r\n (10054) (SQLExecDirectW)')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-18-a3cb542363a4>", line 87, in TimeSeriesPullEMA
    t2.to_sql(datePeriod + ' Day EMA', engine, if_exists='append', index=False)
  File

89
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
90
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
91
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
92
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
93
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])


Exception in thread Thread-563943:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\engine\base.py", line 1182, in _execute_context
    context)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 470, in do_execute
    cursor.execute(statement, parameters)
pyodbc.Error: ('08S01', '[08S01] [Microsoft][ODBC Driver 13 for SQL Server]TCP Provider: An existing connection was forcibly closed by the remote host.\r\n (10054) (SQLExecDirectW)')

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\ProgramData\Anaconda3\lib\threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-18-a3cb542363a4>", line 87, in TimeSeriesPullEMA
    t2.to_sql(datePeriod + ' Day EMA', engine, if_exists='append', index=False)
  File

94
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
dict_keys(['Meta Data', 'Technical Analysis: EMA'])
